# Compute the DMD directly and using FJLT

In [ ]:
from tqdm import tqdm
import sys
import gc
sys.path.append('../../../utils')
from TurboFJLT import *
from TurboFJLT_helpers import FJLT, TurboHDF5Reader
%config InlineBackend.figure_format='retina'

In [ ]:
datafile = "../../../data/fine_airfoil_cascade.h5"
reader = TurboHDF5Reader(datafile)
print(reader)

### Extract the data

In [ ]:
num_snapshots = 300
snapshot_sequence = list(range(num_snapshots))

In [ ]:
def extract_dmd(dmd_result):
    num_non_zero_amps = np.count_nonzero(dmd_result.amplitudes)
    print("Non-zero amplitudes: ", num_non_zero_amps)
    amps = np.zeros(num_non_zero_amps, dtype="complex128")
    eigs = np.zeros(num_non_zero_amps, dtype="complex128")
    modes = np.zeros((dmd_result.modes.shape[0], eigs.shape[0]), dtype="complex128")
    count = 0
    for eig, amp, mode in zip(dmd_result.eigs, dmd_result.amplitudes, dmd_result.modes.T):
        if np.abs(amp) != 0:
            amps[count] = amp
            eigs[count] = eig
            modes[:, count] = mode.T
            count += 1
    return amps, eigs, modes

In [ ]:
def formQ(reader, seq_to_extract):
    q_mf = reader.load_meanflow()
    num_dofs = reader.state_dim
    Q = np.zeros((num_dofs, len(list(seq_to_extract))))
    reader.reset_chunked_loading(seq_to_extract, chunks_dim=50)
    for i, _ in enumerate(tqdm(seq_to_extract)):
        Q[:, i] = reader.load_next()-q_mf
    return Q

In [ ]:
def extract_DMD(Q, gamma=600):
    dmd_sol = SpDMD(svd_rank=50,gamma=gamma,rho=1.e4).fit(Q)
    return dmd_sol

In [ ]:
Q = formQ(reader, snapshot_sequence)
dmd_Q = extract_DMD(Q)
del Q
gc.collect()

amps, eigs, modes = extract_dmd(dmd_Q)
with h5.File("../data/direct_dmd.h5", 'w') as f:
    f.create_dataset("/amps", data=amps)
    f.create_dataset("/eigs", data=eigs)
    f.create_dataset("/modes", data=modes)
    # Garbage collection for memory
    del amps
    del eigs
    del modes
    gc.collect()

In [ ]:
def formB(reader, fjlt, seq_to_extract):
    q_mf = reader.load_meanflow()
    B = np.zeros((fjlt.embedding_dim, len(list(seq_to_extract))))
    reader.reset_chunked_loading(seq_to_extract, chunks_dim=50)
    for i, _ in enumerate(tqdm(seq_to_extract)):
        q_ss = reader.load_next()-q_mf
        B[:, i] = applyFJLT(q_ss, fjlt.P, fjlt.s, fjlt.D)
    return B

In [ ]:
def fjlt_dmd(reader, num_linking_snapshots, snapshots_to_extract):
    fjlt = FJLT(reader.state_dim, num_linking_snapshots, 0.01)
    B = formB(reader, fjlt, snapshots_to_extract)
    dmd_B = extract_DMD(B)
    del B
    gc.collect()
    return dmd_B

In [ ]:
num_linking_snapshots = [2, 4, 6, 8, 12, 16, 24, 32]
for n_sp in num_linking_snapshots:
    dmd_B = fjlt_dmd(reader, n_sp, snapshot_sequence)
    amps, eigs, modes = extract_dmd(dmd_B)

    with h5.File("../data/fjlt_dmd_{}_linking_snapshots.h5".format(n_sp), 'w') as f:
        f.create_dataset("/amps", data=amps)
        f.create_dataset("/eigs", data=eigs)
        f.create_dataset("/modes", data=modes)
        # Garbage collection for memory
        del amps
        del eigs
        del modes
        gc.collect()